# <h1 align="center"> THEME 5 - Résolution numérique de systèmes dynamiques </h1>

### 🎯 Objectifs

- Résoudre des ODE linéaires et non-linéaires
- Résoudre un système d'ODE

### 📚 Notions 

- [Exemple 1](#ex1):
- [Exemple 2](#ex2):
- [Exemple 3](#ex3):

Un [lexique](#lexique) avec l'ensemble des fonctions qui ont été vues est disponible à la fin du notebook.

### 🧰 Librairies

- **Scipy**: est une librairie Python open-source utilisée pour le calcul scientifique. SciPy contient des modules pour l'optimisation, l'algèbre linéaire, l'intégration, l'interpolation, les fonctions spéciales, la FFT, le traitement du signal et de l'image, les solveurs ODE et d'autres tâches courantes en sciences et en ingénierie.

### 🔗 Référence

- [Documentation Scipy](https://docs.scipy.org/doc/scipy/reference/index.html#scipy-api)

### ⚙️ Installation

`pip install scipy`

## <a name="ex1"><h2 align="center"> Exemple 1 - Réaction Non-Linéaire </h2></a>

### 📝 Contexte

Soit la réaction suivante:

$$
2 A \rightarrow B
$$

Son équation est une ODE non-linéaire:

$$
\frac{d C_{A}}{d t}=-k C_{A}^{2}
$$

Avec

- $C_{A0}=1 \, mol.L^{-1}$
- $k=2 \, L.mol^{-1}.min^{-1}$

### ⭐ Objectif

Résoudre l'ODE numériquement et tracer la concentration de A entre t=0 et t=10 min avec 26 points.

### 💻 Code

La résolution numérique d'ODEs à conditions initiales avec Scipy est très simple puisque qu'elle necessite qu'une seule fonction: [`solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy-integrate-solve-ivp). Cette fonction est disponible dans le sous-module `scipy.integrate`.

In [ ]:
import numpy as np
import scipy.integrate as spi
import matplotlib.pyplot as plt

L'utilisation de `solve_ivp` necessite 3 arguments obligatoires:

- `fun`: Une fonction qui décrit l'ODE de la forme suivante: $\frac{dy}{dt} = f(t, y)$
- `t_span`: Un intervalle de temps d'intégration fourni avec un tuple: `(t0, tf)`.
- `y0`: La condition initiale de l'équation.

La résolution d'un système d'ODE est vu dans l'exemple 2, cela utilise les mêmes arguments sauf que la fonction prend un vecteur y comme argument et les conditions initiales sont un vecteur.

Notez ici que le pas de temps est déterminé automatiquement mais peut être spécifié en passant à `solve_ivp` un vecteur qui contient les valeurs de temps ou l'on veut évaluer l'ODE avec l'argument (falcultatif) `t_eval`.

In [ ]:
# Définition de la fonction
# --------------------------------------------------
# Le 1er argument doit toujours être t
# Le 2ème argument doit toujours être la variable intégrée
# Le reste des arguments sont considérés comme supplémentaires
# Doit toujours retourner la valeur évaluée (à gauche de l'équation)
def f(t, C, k):
    """
    ODE de la réaction 2A -> B
    """
    dC = -k * C**2
    return dC


# Définition des paramètres
# --------------------------------------------------
t0 = 0  # Temps initial
tf = 10  # Temps final
k = 2  # Débit de la réaction
C0 = 1  # Concentration initiale
vec_t = np.linspace(t0, tf, 26)  # Discrétisation du temps

# Résolution de l'ODE
# --------------------------------------------------
# Rappel: si un tuple ne contient qu'une seule valeur, il faut ajouter une virgule: (x,)
sim1 = spi.solve_ivp(
    f,  # Fonction de l'ODE
    (t0, tf),  # Intervalle de temps
    (C0,),  # Condition initiale
    args=(k,),  # Arguments supplémentaires de la fonction
    t_eval=vec_t,  # (optionnel) Vecteur de temps où l'ODE doit être évaluée
)

Le résultat de la résolution est un objet qui contient plusieurs données comme:

- `nfev`: Le nombre de fois que la fonction a été évaluée.
- `njev`: Le nombre de fois que le Jacobien a été évalué.
- `success`: `True` si le solveur a réussi, `False` sinon.
- `t`: Le vecteur des valeurs de temps où l'ODE a été évaluée.
- `y`: La matrice des valeurs de la solution de l'ODE.

In [ ]:
print(sim1)

Il faut noter que `sim1.y` est une matrice qui contient la solution d'une variable sur chaque ligne. Puisque dans cet exemple on résout une seule équation, alors la matrice est de dimension 1xN et pour accéder à notre solution il faut indexer la première ligne de la matrice.

Enfin, on trace la solution avec Matplotlib.

In [ ]:
plt.plot(sim1.t, sim1.y[0])
plt.xlabel("t (min)")
plt.ylabel(r"$C_{A}$ (mol/L)")
plt.title("Évolution de la concentration de A")
plt.show()

## <a name="ex2"><h2 align="center"> Exemple 2 - Cinématique Michaelis-Menten </h2></a>

### 📝 Contexte

En 1913, les chercheurs Leonor Michaelis et Maud Menten ont proposé un modèle mathématique pour décrire la vitesse d'une réaction enzymatique basée sur sa cinématique. Cette réaction implique qu'une enzyme, E, se lie à un substrat, S, pour former un complexe, ES, qui libère à son tour un produit, P, régénérant l'enzyme d'origine.

$$
\begin{aligned}
\mathrm{E}+\mathrm{S} \underset{k_{r}}{\stackrel{k_{f}}{\rightleftharpoons}} \mathrm{ES} \stackrel{k_{\mathrm{cat}}}{\longrightarrow} \mathrm{E}+\mathrm{P}
\end{aligned}
$$

Où $k_{f}$ (constante de vitesse en avant), $k_{r}$ (constante de vitesse en arrière) et $k_{cat}$ (constante de vitesse catalytique) désignent les constantes de vitesse, les doubles flèches entre S (substrat) et ES (complexe enzyme-substrat) représentent le fait que la liaison enzyme-substrat est un processus réversible, et la flèche unique en avant représente la formation de P (produit).

En appliquant la loi de conservation de la masse on obtient 4 ODEs non-linéaires:

$$
\begin{aligned}
\frac{\mathrm{d}[\mathrm{E}]}{\mathrm{d} t} &=-k_{f}[\mathrm{E}][\mathrm{S}]+k_{r}[\mathrm{ES}]+k_{\mathrm{cat}}[\mathrm{ES}] \\
\frac{\mathrm{d}[\mathrm{S}]}{\mathrm{d} t} &=-k_{f}[\mathrm{E}][\mathrm{S}]+k_{r}[\mathrm{ES}] \\
\frac{\mathrm{d}[\mathrm{ES}]}{\mathrm{d} t} &=k_{f}[\mathrm{E}][\mathrm{S}]-k_{r}[\mathrm{ES}]-k_{\mathrm{cat}}[\mathrm{ES}] \\
\frac{\mathrm{d}[\mathrm{P}]}{\mathrm{d} t} &=k_{\mathrm{cat}}[\mathrm{ES}]
\end{aligned}
$$

### ⭐ Objectif

En prenant:
- $k_{f} = 10^{-3} \, mol.s^{-1}$
- $k_{r} = 10^{-4} \, mol.s^{-1}$
- $k_{cat} = 0.1 \, mol.s^{-1}$

Et comme conditions initiales: 
- $E_{0} = 200$ 
- $S_{0} = 500$
- $ES_{0} = 0$
- $P_{0} = 0$

Résoudre le système d'ODEs numériquement et tracer la concentration de chaque espèce entre t=0 et t=50s. 

### 💻 Code

La résolution du système est assez similaire à l'exemple 1 à quelques exceptions pour accommoder le fait que nous sommes en train de résoudre un système d'ODEs cette fois.

- Le deuxième argument de notre fonction doit être un vecteur qui contient chaque variables du système.
- La fonction doit retourner un vecteur qui contient les résultats.

In [ ]:
# Définition de la fonction
# --------------------------------------------------
def reaction_enzymatique(t, Y, k_f, k_r, k_cat):
    """
    Système de la réaction enzymatique
    Y: vecteur des concentrations -> [E,S,ES,P]
    """
    dy = [0] * 4  # Création d'un vecteur de zéros [E,S,ES,P]
    dy[0] = -k_f * Y[0] * Y[1] + (k_r + k_cat) * Y[2]  # E
    dy[1] = -k_f * Y[0] * Y[1] + k_r * Y[2]  # S
    dy[2] = k_f * Y[0] * Y[1] - (k_r + k_cat) * Y[2]  # ES
    dy[3] = k_cat * Y[2]  # P
    return dy


# Définition des paramètres
# --------------------------------------------------
k_f = 1e-3
k_r = 1e-4
k_cat = 0.1
k = (k_f, k_r, k_cat)
t = np.linspace(0, 50, 100)
E0 = 200
S0 = 500
y0 = (E0, S0, 0, 0)  # [E,S,ES,P]

# Résolution de l'ODE
# --------------------------------------------------
sim2 = spi.solve_ivp(reaction_enzymatique, t_span=(0, 50), y0=y0, args=k)

In [ ]:
plt.plot(sim2.t, sim2.y[0], label="E")
plt.plot(sim2.t, sim2.y[1], label="S")
plt.plot(sim2.t, sim2.y[2], label="ES")
plt.plot(sim2.t, sim2.y[3], label="P")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
sim2